# HMI for wood harvesting scenario 
This is a KWH HMI which allows to receive S3I-B namedEvents from the forwarder and the harvester. Just run the folliwng code to launch this notebook. 

In [ ]:
import s3i
import getpass
import jwt
import json
import requests
from tools import print_with_timestamp

In [ ]:
HMI_ID = "s3i:359f3c1b-390d-4dd3-ba78-d894c9343a16"
HMI_SECRET = "4ac4f8d7-2e16-4fc1-9158-921f13bb5498"

HARVESTER_ID = "s3i:f7b0057a-6602-480f-ad59-e090868ccd1d"
FORWARDER_ID = "s3i:b31df67f-9d3e-4e93-ab3e-869cde7f3c3f"

# Configure the notebook 

In [ ]:
print_with_timestamp("HMI for the wood harvesting scenario, please log in.")
username = input('[S3I]: Please enter your username:').strip('," ')
password = getpass.getpass('[S3I]: Please enter the password:')
s3i_identity_provider = s3i.IdentityProvider(grant_type='password', 
                                             identity_provider_url="https://idp.s3i.vswf.dev/",
                                             realm='KWH',
                                             client_id=HMI_ID,
                                             client_secret=HMI_SECRET,
                                             username=username,
                                             password=password)

"""
get access token
"""
access_token = s3i_identity_provider.get_token(s3i.TokenType.ACCESS_TOKEN)

"""
decode the access token
"""
parsed_username = jwt.decode(access_token, verify=False)["preferred_username"]
print_with_timestamp("Token received, " + parsed_username + " logged in.")

# Configure the event queue
In order to listen to the namedEvents emitted from the harvester and forwarder, you need to configure your event queue using S3I Config REST API

In [ ]:
print_with_timestamp("Configure the event queue via Config API")
queue_length = input("[S3I]: Please enter the length of your event queue: ")
topic = input("[S3I]: Please enter the event topic(s), Examples: \n - s3i:123.topic \n - ['s3i:123.topic1', 's3i:123.topic2']\n")
request_body = {"queue_length": int(queue_length), "topic": topic}
headers = {'Content-Type': 'application/json', 'Authorization': 'Bearer ' + access_token}
response = requests.post(url="https://config.s3i.vswf.dev/things/{}/broker/event".format(HMI_ID),
                        data=json.dumps(request_body), headers=headers)
print_with_timestamp(response.text)
queue_name = response.json()["queue_name"]

# Listen to the event

In [ ]:

def receive():    
    response = requests.get(url="https://broker.s3i.vswf.dev/"+queue_name , headers=headers)
    if response.status_code == 200:
        if response.content:
            namedEvent = response.json()
            if namedEvent["messageType"] == "eventMessage":
                if namedEvent["sender"] == HARVESTER_ID:
                    sender = "harvester"
                elif namedEvent["sender"] == FORWARDER_ID:
                    sender = "forwarder"
                else:
                    sender = namedEvent["sender"]
                print_with_timestamp("Receive a namedEvent from {}: event generated at {}, content of event: {}".format(sender, namedEvent["timestamp"], namedEvent["content"]))
        else:
            pass
    else:
        print_with_timestamp("Error with status code: {}".format(response.status_code))

while True:
    try:
        receive()
    except KeyboardInterrupt:
        """
        The connection in this application is built not directly via Pika, because Pika Connection (Blocking Connection or Selection Connection) is not allowed to use in Jupyter notebook.
        The connection via Broker API leads to that the event queue will not be automatically deleted, since there is no consumer who listens to the event queue with a callback function
        In order to delete it, we use the Config API. 
        """
        response = requests.delete(url="https://config.s3i.vswf.dev/things/{}/broker/event".format(HMI_ID),
                                  headers=headers)
        if response.status_code == 204:
            print_with_timestamp("Event Queue deleted")
        break